In [ ]:
print('notebook')
!pip install rasterio

In [ ]:
%pip install pandas
%pip install surprise
%pip install geopy
%pip install pyproj
%pip install scikit-surprise

<h1>Segments transform </h1>

In [ ]:
import pandas as pd
import numpy as np
folder="./"
file_path = folder+"seg_all.csv"
df = pd.read_csv(file_path, delimiter=',')
df

In [ ]:
df=df[df['time_1'].isna()==False]
df

In [ ]:
df

In [ ]:
from datetime import datetime
from geopy.distance import geodesic
#from pandarallel import pandarallel
import pandas as pd

# Inicijalizacija pandarallel (koristi sve dostupne jezgre)
#pandarallel.initialize(progress_bar=True)

# Funkcija za računanje vrednosti po redu
def process_row(row):
    try:
        point1 = (row['lat_1'], row['lon_1'])
        point2 = (row['lat_2'], row['lon_2'])

        # Parsiranje vremena
        if 'Z' in row['time_1']:
            time1 = datetime.fromisoformat(row['time_1'].replace("Z", "+00:00"))
            time2 = datetime.fromisoformat(row['time_2'].replace("Z", "+00:00"))
        else:
            time1 = datetime.fromisoformat(row['time_1'])
            time2 = datetime.fromisoformat(row['time_2'])

        # Razlika u nadmorskoj visini
        elev_diff = row['elev_2'] - row['elev_1']

        # Izračunaj rastojanje (u metrima) i vreme (u sekundama)
        distance = geodesic(point1, point2).meters
        time_diff = (time2 - time1).total_seconds()

        # Izračunaj brzinu (m/s) i nagib (%)
        speed = distance / time_diff if time_diff > 0 else 0
        slope = (elev_diff / distance) * 100 if distance > 0 else 0

        return pd.Series([distance, time_diff, speed, slope])

    except Exception as e:
        print(e, row['time_1'])
        return pd.Series([None, None, None, None])

# Paralelna obrada podataka
df[['distance_m', 'time_s', 'speed_m_s', 'slope_percent']] = df.apply(process_row, axis=1)


In [ ]:
import pandas as pd
from geopy.distance import geodesic
from datetime import datetime




# Helper function to calculate speed (m/s)
def calculate_speed(dist_meters, time_seconds):
    return dist_meters / time_seconds if time_seconds > 0 else 0

# Helper function to calculate slope
def calculate_slope(elev_diff, dist_meters):
    return (elev_diff / dist_meters) * 100 if dist_meters > 0 else 0

# Add calculated columns
distances = []
times = []
speeds = []
slopes = []

for index, row in df.iterrows():
    # Get coordinates and timestamps
    point1 = (row['lat_1'], row['lon_1'])
    point2 = (row['lat_2'], row['lon_2'])
    try:
      if 'Z' in row['time_1']:

        time1 = datetime.fromisoformat(row['time_1'].replace("Z", "+00:00"))
        time2 = datetime.fromisoformat(row['time_2'].replace("Z", "+00:00"))
      else :
        time1 = datetime.fromisoformat(row['time_1'])
        time2 = datetime.fromisoformat(row['time_2'])

    except Exception as e:

      print(e,row['time_1'],index)
      time1=datetime.fromisoformat("2012-10-12")
      time2=datetime.fromisoformat("2012-10-12")


    elev_diff = row['elev_2'] - row['elev_1']

    # Calculate distance (meters) and time (seconds)
    distance = geodesic(point1, point2).meters
    time_diff = (time2 - time1).total_seconds()

    # Calculate speed (m/s) and slope (%)
    speed = calculate_speed(distance, time_diff)
    #slope = calculate_slope(elev_diff, distance)

    # Append results
    distances.append(distance)
    times.append(time_diff)
    speeds.append(speed)
    #slopes.append(slope)

# Add results to the DataFrame
df['distance_m'] = distances
df['time_s'] = times
df['speed_m_s'] = speeds
#df['slope_percent'] = slopes

# Save the results to a CSV file


In [ ]:
df.describe()

In [ ]:
df=df[df['time_s']>0]
df=df[df['time_s']<60]
df=df[df['distance_m']<50]
df=df[df['distance_m']>0]


df

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt
#print(df['time_s'].value_counts())
plt.scatter(df['time_s'].value_counts().index,df['time_s'].value_counts())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
#print(df['time_s'].value_counts())

plt.scatter(df['distance_m'].value_counts().index,df['distance_m'].value_counts())

In [ ]:
df=df[df['speed_m_s']<50]
plt.scatter(df['d_from_start'],df['speed_m_s'])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
#print(df['time_s'].value_counts())
plt.scatter(df['speed_m_s'].value_counts().index,df['speed_m_s'].value_counts())

In [ ]:

df['speed_m_s'].describe()

#plt.scatter(df['time_s'],df['speed_m_s'])

In [ ]:
df=df[df['distance_m']<50]
df['distance_m'].describe()

In [ ]:
g1=df.groupby('origname')[['speed_m_s']].agg(['mean','median','std']).reset_index()

#g1[g1['mean']>15]
nevalja=[]
for  n in g1[g1['speed_m_s']['median']>20]['origname'] :
    print(n)
    nevalja.append(n)


In [ ]:
for s in nevalja:
  df=df[df['origname']!=s]


In [ ]:
df

In [ ]:
from pyproj import Transformer

trans = Transformer.from_crs(
    "epsg:4326",
  #  "+proj=utm +zone=10 +ellps=WGS84",
  "epsg:3765",
    always_xy=True,
)
#32610
xx, yy = trans.transform(df["lon_1"].values, df["lat_1"].values)
xx2, yy2 = trans.transform(df["lon_2"].values, df["lat_2"].values)

df["N1"] = xx
df["E1"] = yy
df["N2"] = xx2
df["E2"] = yy2
import math
frameForGroup=df.copy()
num_round=-1
def distance_utm(row):
  n1=row.lat_1
  e1=row.lon_1
  n2=row.lat_2
  e2=row.lon_2
  x=n2-n1
  y=e2-e1
  c=math.sqrt(x*x+y*y)
  return c
def calc_slope(e_d,d_D):
  if e_d==0:
    return 0
  if d_D==0:
    return 0
  return e_d/(d_D*1000)
frameForGroup['lat_1']=round(frameForGroup['N1'], num_round)
frameForGroup['lon_1']=round(frameForGroup['E1'], num_round)
frameForGroup['lat_2']=round(frameForGroup['N2'], num_round)
frameForGroup['lon_2']=round(frameForGroup['E2'], num_round)
frameForGroup['dist_utm']=frameForGroup.apply(distance_utm,axis=1)
#frame['angle_utm']=frame.apply (lambda row: calc_slope(row['elev_d'],row['dist_utm']) , axis=1) # df['elev_d']/(df['dist_2d_wgs']*1000)

frameForGroup['speed_utm']=frameForGroup['dist_utm']/frameForGroup['time_s']
#frameForGroup['angle_utm']=frameForGroup['elev_d']/frameForGroup['dist_utm']
#frameForGroup['angle_utm']=frameForGroup.apply (lambda row: calc_slope(row['elev_d'],row['dist_utm']) , axis=1) # df['elev_d']/(df['dist_2d_wgs']*1000)

#frameForGroup=frameForGroup[frameForGroup['time_d_s']<50]
frameForGroup=frameForGroup[frameForGroup['speed_utm']<10]
#frameForGroup['latlon']=frameForGroup.apply (lambda row: str(row['lat_1']).strip()+'_'+str(row['lon_1']).strip()+'_'+str(row['lat_2']).strip()+'_'+str(row['lon_2']).strip() , axis=1)


frameForGroup['latlon']=frameForGroup.apply (lambda row: str(row['lat_1']).strip()+'_'+str(row['lon_1']).strip()  , axis=1)

#frameForGroup['latlon']=frameForGroup.apply (lambda row: str(row['lat_1']).strip()+'_'+str(row['lon_1']).strip()+'_'+str(row['lat_2']).strip()+'_'+str(row['lon_2']).strip() , axis=1)
frameForGroup["origname_encoded"] = frameForGroup["origname"].astype("category").cat.codes


# Normalize frequencies to avoid dominance of this feature in PCA
frameForGroup['latlon_encoded'] = frameForGroup["latlon"].astype("category").cat.codes

frameForGroup.describe()

In [ ]:
frameForGroup['latlon'].value_counts()[frameForGroup['latlon'].value_counts()>5]

In [ ]:
grouped_frame=frameForGroup[['lat_1', 'lon_1',   'lat_2', 'lon_2','distance_m', 'time_s', 'speed_m_s',
       'slope_percent',    'origname_encoded', 'N1', 'E1', 'N2',
       'E2', 'dist_utm','latlon_encoded']].groupby(['latlon_encoded','origname_encoded']).agg('median')
dd=grouped_frame[[ 'distance_m', 'time_s', 'speed_m_s', 'slope_percent' ]].apply(lambda x: x)

dd=dd.reset_index()

col_sel=['elev_1','distance_m', 'time_s', 'speed_m_s',
       'slope_percent', 'curvature_rad']

col_Sel2=[]
numerical_data=dd[['latlon_encoded','origname_encoded','speed_m_s']]
numerical_data.to_csv('numerical_data.csv')

<h1>frameforgroup</h1>
>>frameforgroup

In [ ]:
import pandas as pd
frameForGroup=pd.read_csv('frameForGroup-2.csv')
frameForGroup[['latlon','latlon_encoded']]
grouped_frame=frameForGroup[['lat_1', 'lon_1',   'lat_2', 'lon_2','distance_m', 'time_s', 'speed_m_s',
        'origname_encoded', 'N1', 'E1', 'N2',
       'E2', 'latlon_encoded']].groupby(['latlon_encoded','origname_encoded']).agg('median')
dd=grouped_frame[[ 'distance_m', 'time_s', 'speed_m_s' ]].apply(lambda x: x)

dd=dd.reset_index()
numerical_data=dd[['latlon_encoded','origname_encoded','speed_m_s']]
#numerical_data=frameForGroup[['origname_encoded','latlon_encoded','speed_m_s']]

In [ ]:
#numerical_data['latlon_encoded'][numerical_data['latlon_encoded']==.value_counts()==1]

In [ ]:
frameForGroup[['latlon','latlon_encoded']]
frameForGroup['latlon_encoded'].unique()
latlon_dict = frameForGroup.set_index('latlon_encoded')['latlon'].to_dict()
lat_dict=frameForGroup.set_index('latlon_encoded')['lat_1'].to_dict()
lon_dict=frameForGroup.set_index('latlon_encoded')['lon_1'].to_dict()
N1_dict=frameForGroup.set_index('latlon_encoded')['N1'].to_dict()
E1_dict=frameForGroup.set_index('latlon_encoded')['E1'].to_dict()
frameForGroup

<h1>NMF</h1>

In [ ]:
numerical_data.describe()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import NMF,non_negative_factorization
from sklearn.metrics import mean_squared_error
ratings_dict = {
    "user": numerical_data['origname_encoded'].values,
    "item": numerical_data['latlon_encoded'].values,
    "rating": numerical_data['speed_m_s'].values
}

df = pd.DataFrame(ratings_dict)
user_item_matrix = df.pivot(index="user", columns="item", values="rating").fillna(0)
#print(user_item_matrix)

nmf = NMF(n_components=1, init='nndsvd', random_state=42)

# Factorize user-item matrix into W (Users) and H (Items)
W_nmf = nmf.fit_transform(user_item_matrix)  # User features
H_nmf = nmf.components_  # Item features

#print("User Features Matrix (W):\n", W)
#print("Item Features Matrix (H):\n", H)


In [ ]:
df.describe()

In [ ]:
user_item_matrix.shape
df.shape[0]/user_item_matrix.size

<h2>evaluate nmf</h2>

In [ ]:
reconstructed_ratings = np.dot(W_nmf, H_nmf)
reconstructed_ratings
user_item_matrix[user_item_matrix>0]
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(user_item_matrix, reconstructed_ratings))
print(f"RMSE: {rmse:.4f}")

<h1>non_negative_factorization</h1>

In [ ]:
W_non, H_non, n_iter = non_negative_factorization(user_item_matrix, n_components=1, init='random', random_state=42)


In [ ]:
reconstructed_ratings = np.dot(W_non, H_non)
reconstructed_ratings
user_item_matrix[user_item_matrix>0]
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(user_item_matrix, reconstructed_ratings))
print(f"RMSE: {rmse:.4f}")

<h1>SVD s</h1>

In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds

# ✅ Example Dataset
 

# Create a sample dataset (user_id, item_id, rating)
 

# ✅ 1. Create User-Item Matrix
#user_item_matrix = df_train.pivot(index="user_id", columns="item_id", values="rating").fillna(0)

# ✅ 2. Apply SVD (Truncated SVD for matrix factorization)
n_factors = 1  # Number of latent factors
new_var = user_item_matrix.values
U, S, Vt = svds(new_var, k=n_factors)

# Convert singular values into a diagonal matrix
S_diag = np.diag(S)

# ✅ 3. Reconstruct the matrix
reconstructed_matrix = np.dot(np.dot(U, S_diag), Vt)

# ✅ 4. Function to Predict Ratings
def predict_rating(user, item):
    """
    Predict rating for a given user and item.
    :param user: user index
    :param item: item index
    :return: predicted rating
    """
    if user < reconstructed_matrix.shape[0] and item < reconstructed_matrix.shape[1]:
        return reconstructed_matrix[user, item]
    else:
        return np.nan  # Return NaN if out of bounds

# Example prediction for user 0 and item 10
predicted_rating = predict_rating(0, 10)
print(f"Predicted rating for user 0 and item 10: {predicted_rating:.2f}")


<h2>evaluate svd</h2>

In [ ]:
reconstructed_ratings = np.dot(U, Vt)
reconstructed_ratings
user_item_matrix[user_item_matrix>0]
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(user_item_matrix, reconstructed_ratings))
print(f"RMSE: {rmse:.4f}")

<h1>TruncatedSVD</h1>

In [ ]:
from sklearn.decomposition import TruncatedSVD

 

svd = TruncatedSVD(n_components=1)
user_factors = svd.fit_transform(user_item_matrix)  # User embeddings
item_factors = svd.components_.T  # Item embeddings (transposed)

print("User Factors (Embeddings):")
print(user_factors.shape)

print("Item Factors (Embeddings):")
print(item_factors.shape)

In [ ]:
user_item_matrix.shape

In [ ]:
reconstructed_ratings = np.dot(user_factors, item_factors.T)
actual_ratings = user_item_matrix[user_item_matrix > 0]
predicted_ratings = reconstructed_ratings[user_item_matrix > 0]
rmse = np.sqrt(mean_squared_error(user_item_matrix, reconstructed_ratings))

print(f"RMSE: {rmse:.4f}")

In [ ]:
print(reconstructed_ratings.shape,actual_ratings.shape)

In [ ]:
from sklearn.decomposition import fastica
K, W, S = fastica(user_item_matrix, n_components=1, random_state=0, whiten='unit-variance')
u_factors=S
i_factors=K 
i_factors.shape

In [ ]:
reconstructed_ratings = np.dot(u_factors, i_factors)
actual_ratings = user_item_matrix[user_item_matrix > 0]
predicted_ratings = reconstructed_ratings[user_item_matrix > 0]
rmse = np.sqrt(mean_squared_error(user_item_matrix, reconstructed_ratings))

print(f"RMSE: {rmse:.4f}")

<h1>SGD </h1>

In [ ]:
import numpy as np
import pandas as pd

class SGDMatrixFactorization:
    def __init__(self, num_users, num_items, n_factors=10, learning_rate=0.01, reg_param=0.02, n_epochs=20):
        """
        Initialize the matrix factorization model using SGD.
        """
        self.num_users = num_users
        self.num_items = num_items
        self.n_factors = n_factors
        self.learning_rate = learning_rate
        self.reg_param = reg_param
        self.n_epochs = n_epochs

        # Initialize user and item latent factors randomly
        self.user_factors = np.random.normal(scale=0.1, size=(num_users, n_factors))
        self.item_factors = np.random.normal(scale=0.1, size=(num_items, n_factors))

    def train(self, df_train):
        """
        Train the model using stochastic gradient descent (SGD).
        :param df_train: Pandas DataFrame with columns ['user_id', 'item_id', 'rating']
        """
        train_data = list(zip(df_train['user'], df_train['item'], df_train['rating']))  # Convert DF to list

        for epoch in range(self.n_epochs):
            np.random.shuffle(train_data)  # Shuffle training data for stochastic updates
            
            for user, item, rating in train_data:
                # Predict rating
                pred = np.dot(self.user_factors[user], self.item_factors[item])
                error = rating - pred

                # Update user and item latent factors
                self.user_factors[user] += self.learning_rate * (error * self.item_factors[item] - self.reg_param * self.user_factors[user])
                self.item_factors[item] += self.learning_rate * (error * self.user_factors[user] - self.reg_param * self.item_factors[item])

            # Compute RMSE after each epoch
            rmse = self.compute_rmse(df_train)
            print(f"Epoch {epoch + 1}/{self.n_epochs}, RMSE: {rmse:.4f}")

    def compute_rmse(self, df):
        """
        Compute RMSE on given dataset.
        """
        errors = [(rating - np.dot(self.user_factors[user], self.item_factors[item])) ** 2 
                  for user, item, rating in zip(df['user'], df['item'], df['rating'])]
        return np.sqrt(np.mean(errors))

    def predict(self, user, item):
        """
        Predict a rating for a given user-item pair.
        """
        return np.dot(self.user_factors[user], self.item_factors[item])

    def full_matrix(self):
        """
        Reconstruct the full user-item rating matrix.
        """
        return np.dot(self.user_factors, self.item_factors.T)


# Example Usage with DataFrame
np.random.seed(42)

# Simulated user-item-rating data
num_users = 100  # Total number of users
num_items = 500 
model = SGDMatrixFactorization(user_item_matrix.shape[0], user_item_matrix.shape[1], n_factors=1, learning_rate=0.01, reg_param=0.02, n_epochs=20)
model.train(df)

# Make a prediction for user 0 and item 10
predicted_rating = model.predict(user=0, item=10)
print(f"Predicted rating for user 0 and item 10: {predicted_rating:.2f}")

<h2>evaluate</h2>

In [ ]:
model.item_factors[:,0]
model.user_factors[:,0]


reconstructed_ratings = np.dot(model.user_factors, model.item_factors.T)
actual_ratings = user_item_matrix[user_item_matrix > 0]
predicted_ratings = reconstructed_ratings[user_item_matrix > 0]
rmse = np.sqrt(mean_squared_error(user_item_matrix, reconstructed_ratings))

print(f"RMSE: {rmse:.4f}")

<h1>ALS</h1>

In [ ]:
import numpy as np
import pandas as pd

class ALS:
    def __init__(self, num_users, num_items, n_factors=10, reg_param=0.02, n_epochs=20):
        """
        Alternating Least Squares (ALS) Matrix Factorization for Collaborative Filtering
        :param num_users: Number of unique users
        :param num_items: Number of unique items
        :param n_factors: Number of latent factors
        :param reg_param: Regularization parameter to avoid overfitting
        :param n_epochs: Number of epochs to run ALS
        """
        self.num_users = num_users
        self.num_items = num_items
        self.n_factors = n_factors
        self.reg_param = reg_param
        self.n_epochs = n_epochs

        # Initialize the user and item matrices with small random values
        self.user_factors = np.random.normal(scale=0.1, size=(num_users, n_factors))
        self.item_factors = np.random.normal(scale=0.1, size=(num_items, n_factors))

    def train(self, df):
        """
        Train the ALS model on a DataFrame containing user-item-rating interactions.
        :param df: Pandas DataFrame containing columns ['user_id', 'item_id', 'rating']
        """
        # Convert DataFrame to a list of tuples (user_id, item_id, rating)
        train_data = list(zip(df['user_id'], df['item_id'], df['rating']))

        for epoch in range(self.n_epochs):
            # Alternating updates
            for user, item, rating in train_data:
                # Update user matrix
                self.user_factors[user] = self.update_user_factors(user, item, rating)
                # Update item matrix
                self.item_factors[item] = self.update_item_factors(user, item, rating)

            # Optionally compute RMSE or other evaluation metrics
            rmse = self.compute_rmse(df)
            print(f"Epoch {epoch+1}/{self.n_epochs}, RMSE: {rmse:.4f}")

    def update_user_factors(self, user, item, rating):
        """
        Update user latent factors for a specific user-item-rating triplet
        :param user: user index
        :param item: item index
        :param rating: actual rating
        """
        item_factors = self.item_factors[item]
        predicted_rating = np.dot(self.user_factors[user], item_factors)
        error = rating - predicted_rating

        # Regularized update rule
        self.user_factors[user] += self.reg_param * error * item_factors
        return self.user_factors[user]

    def update_item_factors(self, user, item, rating):
        """
        Update item latent factors for a specific user-item-rating triplet
        :param user: user index
        :param item: item index
        :param rating: actual rating
        """
        user_factors = self.user_factors[user]
        predicted_rating = np.dot(user_factors, self.item_factors[item])
        error = rating - predicted_rating

        # Regularized update rule
        self.item_factors[item] += self.reg_param * error * user_factors
        return self.item_factors[item]

    def compute_rmse(self, df):
        """
        Compute RMSE on the given dataset.
        :param df: DataFrame containing ['user_id', 'item_id', 'rating']
        """
        errors = [(rating - np.dot(self.user_factors[user], self.item_factors[item])) ** 2 
                  for user, item, rating in zip(df['user_id'], df['item_id'], df['rating'])]
        return np.sqrt(np.mean(errors))

    def predict(self, user, item):
        """
        Predict a rating for a given user-item pair.
        :param user: user index
        :param item: item index
        """
        return np.dot(self.user_factors[user], self.item_factors[item])

    def full_matrix(self):
        """
        Reconstruct the full user-item rating matrix.
        """
        return np.dot(self.user_factors, self.item_factors.T)

# Example usage with a DataFrame
np.random.seed(42)

# Example DataFrame
num_users = user_item_matrix.shape[0]  # Total number of users
num_items = user_item_matrix.shape[1]  # Total number of items

ratings_dict = {
    "user": numerical_data['origname_encoded'].values,
    "item": numerical_data['latlon_encoded'].values,
    "rating": numerical_data['speed_m_s'].values
}


df_train = pd.DataFrame({
    'user_id': ratings_dict['user'],#np.random.randint(0, num_users, 10000),
    'item_id': ratings_dict['item'],#np.random.randint(0, num_items, 10000),
    'rating': ratings_dict['rating']#np.random.randint(1, 6, 10000)  # Ratings between 1 and 5
})

# Train the ALS model
model_als = ALS(num_users, num_items, n_factors=1, reg_param=0.02, n_epochs=20)
model_als.train(df_train)

# Make a prediction for user 0 and item 10
predicted_rating = model_als.predict(user=0, item=10)
print(f"Predicted rating for user 0 and item 10: {predicted_rating:.2f}")


In [ ]:
model_als.item_factors[:,0]
model_als.user_factors[:,0]


reconstructed_ratings = np.dot(model_als.user_factors, model_als.item_factors.T)
actual_ratings = user_item_matrix[user_item_matrix > 0]
predicted_ratings = reconstructed_ratings[user_item_matrix > 0]
rmse = np.sqrt(mean_squared_error(user_item_matrix, reconstructed_ratings))

print(f"RMSE: {rmse:.4f}")

In [ ]:
user_item_matrix

<h1> make tiiff</h1>

In [ ]:
#print(model.item_factors[:0].shape)
user_item_matrix.columns
Vt[0,:]

In [ ]:
#vals=model.item_factors[:,0]
#output_file = "grid_mapALS.tif"

vals=Vt[0,:]
output_file = "grid_mapSVD.tif"
# vals=H_nmf[:,0]
# output_file = "grid_mapNMF.tif"

# vals=item_factors[:,0]
# output_file = "grid_mapTrunc_SVD.tif"

# vals=i_factors[:,0]
# output_file = "grid_mapTrunc_fastica.tif"

# vals=model.item_factors[:,0]
# output_file = "grid_mapSGD.tif"

# vals=model_als.item_factors[:,0]
# output_file = "grid_mapALS.tif"

frameForTiff =pd.DataFrame(data={
    'item_code':user_item_matrix.columns,
 #   'value':item_factors[:,0]*1000,
# 'value':H.T[:,0]*1000,
'value':vals,
    'latlon':[latlon_dict.get(x) for x in user_item_matrix.columns ]
})
frameForTiff[['lat1', 'lon1']] = frameForTiff['latlon'].str.split('_', expand=True).astype(float)


frameForTiff["lon"] = [lon_dict.get(x) for x in user_item_matrix.columns ]
frameForTiff["lat"] = [lat_dict.get(x) for x in user_item_matrix.columns ]
frameForTiff["N1"] = [N1_dict.get(x) for x in user_item_matrix.columns ]
frameForTiff["E1"] = [E1_dict.get(x) for x in user_item_matrix.columns ]
frameForTiff

In [ ]:
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import from_origin

# ✅ 1. Load Data (Example DataFrame)
# Replace with actual DataFrame loading
df = pd.DataFrame({
    'lon': frameForTiff['lat1'] ,  # X-coordinates in EPSG:3765
    'lat': frameForTiff['lon1'] ,  # Y-coordinates in EPSG:3765
    'value': frameForTiff['value']  # Example values with NaN
})

# ✅ 2. Define Grid Extent & Resolution
resolution = 100  # Set grid cell size in meters (adjust as needed)

# Compute grid boundaries
min_x, max_x = df['lon'].min(), df['lon'].max()
min_y, max_y = df['lat'].min(), df['lat'].max()

# Compute grid size
cols = int((max_x - min_x) / resolution) + 1
rows = int((max_y - min_y) / resolution) + 1

# ✅ 3. Create Empty Grid with NaN
grid = np.full((rows, cols), np.nan, dtype=np.float32)

# ✅ 4. Map DataFrame Values to Grid
for _, row in df.iterrows():
    col_idx = int((row['lon'] - min_x) / resolution)
    row_idx = int((max_y - row['lat']) / resolution)  # Inverted y-axis
    grid[row_idx, col_idx] = row['value']

# ✅ 5. Define GeoTransform
transform = from_origin(min_x, max_y, resolution, resolution)

# ✅ 6. Save as GeoTIFF

with rasterio.open(
    output_file,
    "w",
    driver="GTiff",
    height=rows,
    width=cols,
    count=1,
    dtype=np.float32,
    crs="EPSG:3765",
    transform=transform,
    nodata=np.nan
) as dst:
    dst.write(grid, 1)

print(f"GeoTIFF map saved as '{output_file}'")


In [ ]:
H.T[:,0]

In [ ]:
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import from_origin

# Sample DataFrame with 'value' column


# Split 'latlon' into 'lat' and 'lon'
 
frameForTiff[['lat', 'lon']] = frameForTiff['latlon'].str.split('_', expand=True).astype(float)
 
# Define extent
min_lon, max_lon = frameForTiff['lon'].min(), frameForTiff['lon'].max()
min_lat, max_lat = frameForTiff['lat'].min(), frameForTiff['lat'].max()

# Determine grid resolution (smallest step size)
lon_step = np.min(np.diff(np.sort(frameForTiff['lon'].unique())))
lat_step = np.min(np.diff(np.sort(frameForTiff['lat'].unique())))

# Define grid size
cols = int((max_lon - min_lon) / lon_step) + 1
rows = int((max_lat - min_lat) / lat_step) + 1

# Create an empty raster grid filled with NaN
grid = np.full((rows, cols), np.nan, dtype=np.float32)

# Fill the grid with 'value' from the DataFrame
for _, row in frameForTiff.iterrows():
    col_idx = int((row['lon'] - min_lon) / lon_step)
    row_idx = int((max_lat - row['lat']) / lat_step)  # Invert Y-axis for raster order
    grid[row_idx, col_idx] = row['value']

# Define transformation
transform = from_origin(min_lon, max_lat, lon_step, lat_step)



In [ ]:
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import from_origin

# Sample DataFrame with 'value' column


# Split 'latlon' into 'lat' and 'lon'

 
# Define extent
min_N1, max_N1 = frameForTiff['N1'].min(), frameForTiff['N1'].max()
min_E1, max_E1 = frameForTiff['E1'].min(), frameForTiff['E1'].max()
min_lon, max_lon = frameForTiff['lon'].min(), frameForTiff['lon'].max()
min_lat, max_lat = frameForTiff['lat'].min(), frameForTiff['lat'].max()

# Determine grid resolution (smallest step size)
N1_step = np.min(np.diff(np.sort(frameForTiff['N1'].unique())))
E1_step = np.min(np.diff(np.sort(frameForTiff['E1'].unique())))
print(N1_step)
# Define grid size
cols = int((max_N1 - min_N1) / N1_step) + 1
rows = int((max_E1 - min_E1) / E1_step) + 1

# Create an empty raster grid filled with NaN
grid = np.full((rows, cols), np.nan, dtype=np.float32)

# Fill the grid with 'value' from the DataFrame
for _, row in frameForTiff.iterrows():
    col_idx = int((row['N1'] - min_N1) / N1_step)
    row_idx = int((max_E1 - row['E1']) / E1_step)  # Invert Y-axis for raster order
    grid[row_idx, col_idx] = row['value']

# Define transformation
transform = from_origin(min_lon, max_lat, N1_step, E1_step)



In [ ]:
rows, cols

In [ ]:
with rasterio.open(
    "grid_map4.tif",
    "w",
    driver="GTiff",
    height=rows,
    width=cols,
    count=1,
    dtype=np.float32,
    crs= "epsg:3765",  # Change if needed"epsg:3765"
    transform=transform
) as dst:
    dst.write(grid, 1)

print("GeoTIFF map saved as 'grid_map.tif'")


PCA

In [ ]:
import pandas as pd
from geopy.distance import geodesic
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#!python.exe -m pip install --upgrade pip
#pip install surprise
%pip install --upgrade pip setuptools wheel cython numpy


In [ ]:
numerical_data=pd.read_csv('numerical_data.csv')


<h1>NMF sklearn</h1>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
ratings_dict = {
    "user": numerical_data['origname_encoded'].values,
    "item": numerical_data['latlon_encoded'].values,
    "rating": numerical_data['speed_m_s'].values
}

df = pd.DataFrame(ratings_dict)
print(df)
user_item_matrix = df.pivot(index="user", columns="item", values="rating").fillna(0)
#print(user_item_matrix)

nmf = NMF(n_components=1, init='random', random_state=42)

# Factorize user-item matrix into W (Users) and H (Items)
W = nmf.fit_transform(user_item_matrix)  # User features
H = nmf.components_  # Item features

#print("User Features Matrix (W):\n", W)
#print("Item Features Matrix (H):\n", H)


In [ ]:
numerical_data

In [ ]:
H.size

In [ ]:
nmf = NMF(n_components=1, init='random', random_state=42)

# Factorize user-item matrix into W (Users) and H (Items)
W = nmf.fit_transform(user_item_matrix)  # User features
H = nmf.components_  # Item features
H.T

In [ ]:
user_item_matrix['latent']=H.T
user_item_matrix

In [ ]:
from surprise import SVD
from surprise import Dataset, Reader, accuracy
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV

# Scale represents speed possibilities from 0m/s to 9m/s
reader = Reader(rating_scale=(0, 5))
# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(numerical_data, reader)
data.df
from surprise import accuracy, Dataset, SVD, NMF, SVDpp,BaselineOnly,KNNBasic,SlopeOne,CoClustering
from surprise.model_selection import KFold
from surprise.model_selection import cross_validate

# define a cross-validation iterator
#kf = KFold(n_splits=2,shuffle=True)

#algo = SVD()
algos={
    'svd':SVD(),
    'NMF':NMF(),
    #,'SVDpp':SVDpp(),
   # 'BaselineOnly':BaselineOnly(),
    #'SlopeOne':SlopeOne(),
    'CoClustering':CoClustering()
}
for  algo in algos:
  print(algo)
  #print(cross_validate(algos[algo], data, cv=5))
  cross_validate(algos[algo], data, measures=["RMSE", "MAE"], cv=3, verbose=True)
  print()
  print()




In [ ]:
#%pip install sklearn
#%pip install git+https://github.com/NicolasHug/Surprise.git
#%pip install scikit-surprise
###%pip install --upgrade pip setuptools wheel

!pip install scikit-surprise --no-build-isolation
##%pip install cython


In [ ]:
%pip install scikit-learn

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
ratings_dict = {
    "user": numerical_data['origname_encoded'].values,
    "item": numerical_data['latlon_encoded'].values,
    "rating": numerical_data['speed_m_s'].values
}

df = pd.DataFrame(ratings_dict)
print(df)

In [ ]:
user_item_matrix = df.pivot(index="user", columns="item", values="rating").fillna(0)
#print(user_item_matrix)

nmf = NMF(n_components=2, init='random', random_state=42)

# Factorize user-item matrix into W (Users) and H (Items)
W = nmf.fit_transform(user_item_matrix)  # User features
H = nmf.components_  # Item features

#print("User Features Matrix (W):\n", W)
#print("Item Features Matrix (H):\n", H)


In [ ]:
from sklearn.cluster import SpectralClustering
from sklearn.cluster import KMeans

# Number of clusters
num_clusters = 5
item_features_df = pd.DataFrame(H.T, columns=["Feature 1", "Feature 2"])

# Perform clustering on item features
#spectral_clustering = SpectralClustering(n_clusters=num_clusters, affinity="nearest_neighbors", random_state=42)
#item_clusters = spectral_clustering.fit_predict(H.T)
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
item_clusters = kmeans.fit_predict(H.T)
# Assign clusters to items
item_features_df["Cluster"] = item_clusters


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot item clusters
plt.figure(figsize=(8, 6))
sns.scatterplot(
    x=item_features_df["Feature 1"],
    y=item_features_df["Feature 2"],
    hue=item_features_df["Cluster"],
    palette="viridis",
    s=100
)

# Add item labels
#
plt.xlabel("Latent Feature 1")
plt.ylabel("Latent Feature 2")
plt.title("Item Clusters in Latent Feature Space (NMF)")
plt.legend(title="Cluster")
plt.show()


In [ ]:
%pip install scikit-surprise

In [ ]:
from sklearn.cluster import SpectralClustering
from sklearn.cluster import KMeans

# Number of clusters
num_clusters = 5
euser_features_df = pd.DataFrame(W, columns=["Feature 1", "Feature 2"])

# Perform clustering on item features
#spectral_clustering = SpectralClustering(n_clusters=num_clusters, affinity="nearest_neighbors", random_state=42)
#item_clusters = spectral_clustering.fit_predict(H.T)
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
item_clusters = kmeans.fit_predict(W)
# Assign clusters to items
euser_features_df["Cluster"] = item_clusters
euser_features_df["Cluster"] = item_clusters


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot item clusters
plt.figure(figsize=(8, 6))
sns.scatterplot(
    x=euser_features_df["Feature 1"],
    y=euser_features_df["Feature 2"],
    hue=euser_features_df["Cluster"],
    palette="viridis",
    s=100
)

# Add item labels
#
plt.xlabel("Latent Feature 1")
plt.ylabel("Latent Feature 2")
plt.title("Item Clusters in Latent Feature Space (NMF)")
plt.legend(title="Cluster")
plt.show()


In [ ]:
predicted_matrix = np.dot(W, H)
print("Predicted Ratings Matrix:\n", predicted_matrix)
# Convert to DataFrame for easy comparison
pred_df = pd.DataFrame(predicted_matrix, index=user_item_matrix.index, columns=user_item_matrix.columns)

# Calculate RMSE only for non-zero ratings
true_ratings = user_item_matrix.to_numpy().flatten()
pred_ratings = pred_df.to_numpy().flatten()
mask = true_ratings > 0  # Only compare actual ratings

rmse = np.sqrt(mean_squared_error(true_ratings[mask], pred_ratings[mask]))
print(f"RMSE: {rmse}")


In [ ]:
%pip install tensorflow numpy pandas scikit-learn


In [ ]:
%conda install -c conda-forge scikit-surprise